# Depth Visualizer for Multiple EXR Files

Select multiple `.exr` files and visualize their R, G, B channels and combined RGB in sequence.

In [ ]:
# If running locally (VS Code, Jupyter):
import os
import tkinter as tk
from tkinter import filedialog

import OpenEXR
import Imath
import numpy as np
import matplotlib.pyplot as plt

# File-selection dialog
root = tk.Tk()
root.withdraw()
filepaths = filedialog.askopenfilenames(
    title="Select one or more OpenEXR files",
    filetypes=[("OpenEXR files", "*.exr"), ("All files", "*.*")]
)
root.destroy()

filepaths = list(filepaths)
print(f"Selected {len(filepaths)} files:")
for f in filepaths:
    print(" -", f)


In [ ]:
# Functions to load EXR and visualize
def load_exr(filename):
    """Load R, G, B channels from an EXR into numpy arrays."""
    exr = OpenEXR.InputFile(filename)
    dw = exr.header()["dataWindow"]
    W = dw.max.x - dw.min.x + 1
    H = dw.max.y - dw.min.y + 1
    pt = Imath.PixelType(Imath.PixelType.FLOAT)

    channels = {}
    for c in ("R", "G", "B"):
        buf = exr.channel(c, pt)
        arr = np.frombuffer(buf, dtype=np.float32)
        channels[c] = arr.reshape(H, W)

    return channels

def normalize_channel(arr):
    """Normalize a float32 array for display (0–1)."""
    mn, mx = arr.min(), arr.max()
    return (arr - mn) / (mx - mn + 1e-8)

def depth_linear_to_meters(depth_linear, near=0.1, far=1000.0):
    """Convert 0–1 linear depth to meters."""
    return near * far / (far - (far - near) * depth_linear)


In [ ]:
# Process and visualize
for filepath in filepaths:
    if not os.path.isfile(filepath):
        print(f"File not found: {filepath}")
        continue

    print(f"\nProcessing: {filepath}")
    channels = load_exr(filepath)
    for c in channels:
        print(f" {c} channel shape: {channels[c].shape}")

    # Assume blue channel is depth (matches your Unity shader)
    depth_linear = channels["B"]
    depth_meters = depth_linear_to_meters(depth_linear)

    # Normalize for display
    normalized_depth = normalize_channel(depth_linear)

    # Plotting
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))

    # Raw linear depth (0-1)
    im0 = axs[0].imshow(depth_linear, cmap="plasma")
    axs[0].set_title("Raw Linear Depth (0–1)")
    axs[0].axis("off")
    plt.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)

    # Depth in meters (physical meaning)
    im1 = axs[1].imshow(depth_meters, cmap="plasma")
    axs[1].set_title(f"{os.path.basename(filepath)}\nDepth in Meters\n[{depth_meters.min():.1f}m, {depth_meters.max():.1f}m]")
    axs[1].axis("off")
    plt.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)

    # Normalized colored visualization
    im2 = axs[2].imshow(normalized_depth, cmap="viridis")
    axs[2].set_title("Depth (Viridis Colormap)")
    axs[2].axis("off")
    plt.colorbar(im2, ax=axs[2], fraction=0.046, pad=0.04)

    plt.tight_layout()

    # ----------- CLICK-TO-MEASURE TOOL BELOW -----------
    def onclick(event):
        if event.inaxes in axs:
            x = int(event.xdata)
            y = int(event.ydata)
            if 0 <= x < depth_meters.shape[1] and 0 <= y < depth_meters.shape[0]:
                d = depth_meters[y, x]
                print(f"Clicked at pixel ({x}, {y}) → Distance: {d:.2f} meters")
            else:
                print("Clicked outside the valid image!")

    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    plt.show()
